In [316]:
import pandas as pd
from pandas import *
from pandas.io.json import json_normalize

import json

pd.options.display.max_columns = 50

In [332]:
with open('raw.json') as f:
    data = json.load(f)

In [333]:
questions = json_normalize(data, record_path=['questions'], meta=['country', 'name'])
question_params = json_normalize(data, record_path=['questions', 'params'], meta=[['questions', 'id']])
question_params.columns = ['id', 'questions.id']
question_groups = json_normalize(data, record_path='questionParams')
party_positions = json_normalize(data, record_path=['partyPositions', 'answers'],
               meta=['country', ['partyPositions', 'id'], ['partyPositions', 'name'],
                    ['partyPositions', 'party'], ['partyPositions', 'image']])

In [334]:
question_params.id = question_params.id.apply(lambda x: str(x))
question_groups.id = question_groups.id.apply(lambda x: str(x))
qpg = pd.merge(question_params, question_groups, on='id', how='left')
qpg.value = qpg.value.fillna(qpg['id'])

In [335]:
def map_to_groups(x):
    result = {}

    for k in x.params.keys():
        result[qpg[(qpg['questions.id'] == x.id) & (qpg['id'] == k)].values[0][2] \
               .lower().replace(' ', '_')] = x.params[k]
        
    return result

questions.params = questions.apply(map_to_groups, axis=1)

questions = questions.join(questions.params.apply(pd.Series))

In [357]:
full = pd.merge(party_positions, questions, left_on='question_id', right_on='id')

In [358]:
full = full[[
    u'answer',
    u'question_id',
    u'source',
    u'source_desc',
    u'source_link',
    'country_x',
    'name',
    'partyPositions.id',
    'partyPositions.party',
    'partyPositions.name',
    'partyPositions.image',
    u'value',
    u'economic_liberalisation',
    u'environmental_protection',
    u'european_integration',
    u'law_and_order',
    u'liberal_society',
    u'restrictive_financial_policy',
    u'restrictive_immigration_policy',
    'xy-lc',
    'xy-lr'
]]

full.columns = [
    'answer',
    'question_id',
    'motivation_src',
    'motivation',
    'motivation_src_link',
    'country_code',
    'country_name',
    'party_id',
    'party_name_full',
    'party_name_short',
    'party_image',
    'question',
    'economic_liberalisation',
    'environmental_protection',
    'european_integration',
    'law_and_order',
    'liberal_society',
    'restrictive_financial_policy',
    'restrictive_immigration_policy',
    'xy_lc',
    'xy_lr'  
]

In [386]:
full[full.party_name_full == 'The Workers Party'][['question', 'answer', 'xy_lr']]

,question,answer,xy_lr
2911,Social programmes should be maintained even at...,100.0,-1
2921,The state should provide stronger financial su...,100.0,-1
2931,The European Union should rigorously punish Me...,0.0,0
2941,Asylum-seekers should be distributed proportio...,NaN,0
2951,Immigration into Ireland should be made more r...,25.0,0
2961,Immigrants from outside Europe should be requi...,75.0,0
2971,The legalisation of same sex marriages is a go...,100.0,0
2981,The legalisation of the personal use of soft d...,NaN,0
2991,Euthanasia should be legalised,NaN,0
3001,Government spending should be reduced in order...,0.0,1


In [384]:
full[full.party_id == 386][['question', 'answer', 'european_integration']]

,question,answer,european_integration
2910,Social programmes should be maintained even at...,100.0,0
2920,The state should provide stronger financial su...,100.0,0
2930,The European Union should rigorously punish Me...,75.0,1
2940,Asylum-seekers should be distributed proportio...,NaN,1
2950,Immigration into Ireland should be made more r...,NaN,0
2960,Immigrants from outside Europe should be requi...,NaN,0
2970,The legalisation of same sex marriages is a go...,100.0,0
2980,The legalisation of the personal use of soft d...,NaN,0
2990,Euthanasia should be legalised,NaN,0
3000,Government spending should be reduced in order...,0.0,0


In [374]:
full[['answer', 'party_id', 'party_name_full', 'party_name_short']] \
    .groupby(['party_id', 'party_name_short']) \
    .count().sort_values('answer')



,,answer,party_name_full
party_id,party_name_short,,
502,Koalicja Europejska PO PSL SLD .N ZIELONI,6,22
386,Social Democrats,7,22
520,SAB,8,22
356,Možemo!,10,22
512,Ataka,10,22
314,FP,10,22
274,LSDP,10,22
529,START,10,22
533,DD,10,22
